<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [98]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [214]:
df = pd.DataFrame({
    'x1':[0,0,1,0,1,1,0],
    'x2':[0,1,0,1,0,1,0],
    'x3':[1,1,1,0,0,1,0],
    'y':[0,1,1,1,1,0,0]},
    dtype='int8')
df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [215]:
feats = list(df)[:-1]
target = 'y'
X = df[feats].values
y = np.array([[i] for i in df[target]])

In [216]:
X.shape, y.shape

((7, 3), (7, 1))

In [231]:
class NickNetwork:
    def __init__(self, inputs=3, hiddenlayers=4, outputnodes=1):
        self.inputs = inputs
        self.hiddenlayers = hiddenlayers
        self.outputnodes = outputnodes
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    def sigmoidprime(self, x):
        return x * (1 - x)
    
    def forward(self, X):
        self.hidden_sum = np.dot(X, self.weights1)
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output
    
    def back(self, X, y, o):
        self.error = y - o
        self.delta = self.error * self.sigmoidprime(o)
        self.z2_error = self.delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.sigmoidprime(self.activated_hidden)
        self.weights1 += X.T.dot(self.z2_delta)
        self.weights2 += self.activated_hidden.T.dot(self.delta)
    
    def fit_predict(self, X, y, iter=1000, print_iters=100, verbose=False):
        '''
        Train network over 'iter' number of iterations
        print_iters = Print predicted information every 'print_iters' iterations when
        verbose is True
        verbose = Print information about model, adjust amount with 'print_iters'
        '''
        # Initialize weights
        self.weights1 = np.random.rand(self.inputs, self.hiddenlayers)
        self.weights2 = np.random.rand(self.hiddenlayers, self.outputnodes)

        if verbose:
            for i in range(iter):
                initial = self.forward(X)
                self.back(X,y,initial)
                if (i == 0) or (i % print_iters ==0):
                    print('=' * 9 + f'EPOCH {i}' + '=' * 9 + '\n')
                    print('Input: \n', X)
                    print('Actual Output: \n', y)
                    print('Predicted Output: \n', str(self.forward(X)))
                    print('Loss: \n', str(np.mean(np.square(y - self.forward(X).round(4)))))
        if verbose is False:
            for i in range(iter):
                initial = self.forward(X)
                self.back(X,y,initial)
            print('Number of iterations: \n', str(iter))
            print('Predicted Output: \n', str(self.forward(X).round(4)))

    # WAS WORKING THEN BROKE, NEED MORE TIME TO LOOK AT IT!    
    # def score(self, X, y, iter=100):
    #     '''
    #     Same as fit_predict method but returns an accuracy score instead
    #     '''
    #     for _ in range(iter):
    #         init = self.forward(X)
    #         self.back(X,y,init)
    #     print(f'For {iter} iterations, the accuracy score is: {accuracy_score(self.activated_output, y)}')

In [232]:
nn = NickNetwork()

In [233]:
nn.fit_predict(X,y, iter=100)

Number of iterations: 
 100
Predicted Output: 
 [[0.2789]
 [0.5612]
 [0.5707]
 [0.7847]
 [0.7982]
 [0.6487]
 [0.496 ]]


In [234]:
nn.fit_predict(X,y,iter=1000, print_iters=250, verbose=True)

=========EPOCH 0=========

Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.64510151]
 [0.66087059]
 [0.6564824 ]
 [0.62884198]
 [0.6233613 ]
 [0.67006709]
 [0.60895179]]
Loss: 
 0.2498095828571429
=========EPOCH 250=========

Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.07622632]
 [0.80516272]
 [0.80642211]
 [0.87638518]
 [0.87066793]
 [0.24851358]
 [0.25746869]]
Loss: 
 0.03446977
=========EPOCH 500=========

Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.02431481]
 [0.92294411]
 [0.92344103]
 [0.94048162]
 [0.9397346 ]
 [0.07387554]
 [0.14426009]]
Loss: 
 0.006551785714285714
=========EPOCH 750=========

Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Act

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

In [28]:
np.__version__

'1.17.3'

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [60]:
# input image dimensions
img_rows, img_cols = 28, 28

In [106]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [107]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [108]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [109]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [110]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [176]:
sampler = np.random.randint(60000, size=1000)
sampler

array([50887, 49711, 36515, 46178,  9043, 12626, 50571, 52978,  2257,
       50613, 56493, 11226, 46460, 52091, 49259, 22470, 44688,  6468,
       11634, 58967, 50848, 45812, 28436, 34985, 27146, 11359, 48488,
       46064, 50820, 36073, 40686, 12142, 22905,  8682, 59834, 23419,
       50042,  8556, 31465, 11402, 55364, 47729, 58815, 40857, 23642,
       51139, 41259, 53513, 21067, 54177, 24622, 22969, 26495, 11102,
        9666, 37219, 14740, 29641, 33842, 59693, 16461, 15208, 36093,
       10416, 47129, 41682, 28045, 14273, 42230, 17677, 49985,  3987,
       28669, 23974, 52886, 17207, 28689, 57036,  4677, 39360, 25938,
       52776, 46989, 19562, 50918, 24218, 47225, 23626, 27101, 22975,
        8012, 36424, 32161, 39354, 27454, 37485, 56229, 26410, 29242,
       13241, 39813,  7107, 57193, 55418,  1865, 24046, 36736, 53641,
        6327,  4422, 24674, 39113, 42338, 40491,  8528, 51619, 12038,
        6910, 45791, 58036, 55466, 12701, 33388, 35792, 14312, 47272,
       35081, 13406,

In [177]:
X = x_train[sampler]
y = np.array([[i] for i in y_train[sampler]])

In [212]:
X.shape, y.shape

((1000, 784), (1000, 1))

In [172]:
##### Your Code Here #####
nn = NickNetwork(inputs=784, hiddenlayers=3, outputnodes=1)

In [173]:
nn.fit_predict(X,y, iter=1000, print_iters=250, verbose=True)

=========EPOCH 0=========

Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [[0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]]
Predicted Output: 
 [[2.50284290e-15]
 [2.50284290e-15]
 [2.50284290e-15]
 [2.50284290e-15]
 [2.50284290e-15]
 [2.50284290e-15]
 [2.50284290e-15]
 [2.50284290e-15]
 [2.50284290e-15]
 [2.5028

This output is silly, let's create a new method in the class for this problem

In [192]:
nn = NickNetwork(inputs=784, outputnodes=1)

In [193]:
nn.score(X,y, iter=1000)

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [208]:
y.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?